# NEW YORK TAXI FARE

<img src='img/taxi_fare.PNG' width=2000>

Nesta aula, vamos trabalhar com dados de corridas de taxi em Nova York através de um dataSet consagrado em estudos que vão desde a Caursera e Google Coud até o Kaggle. O objetivo será identificar a série de dados do _fare amount_ (valor da corrida de taxi) como variável taarget, e construir séries de dados que possam explicar estes valores. Para isso, teremos que manipular os dados existentes conforme descrito abaixo, com o intuito de obter uma regressão explicativa para os valores cobrados por este serviço de transporte.

### Descrições de arquivo

- train.csv - Recursos de entrada e valores de fare_amount de destino para o conjunto de treinamento (cerca de 55 milhões de linhas).
- test.csv - Recursos de entrada para o conjunto de testes (cerca de 10 mil linhas).
- sample_submission.csv - um arquivo de envio de amostra no formato correto (keys key e fare_amount). Este arquivo 'prevê' fare_amount para $ 11.35 para todas as linhas, que é a média de fare_amount do conjunto de treinamento.

#### Para baixar os arquivos acesse o link [clicando aqui](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data)

### Campos de dados

1.Variável de Identidade

- key - string única que identifica cada linha nos conjuntos de treinamento e teste.

2.Variáveis Explicativas

- pickup_datetime - valor do registro de data e hora indicando quando o taxi começou.
- pickup_longitude - flutua pela coordenada de longitude de onde o taxi começou.
- pickup_latitude - flutua para a coordenada de latitude de onde o táxi começou.
- dropoff_longitude - flutua pela coordenada de longitude de onde o passeio de táxi terminou.
- dropoff_latitude - flutua para a coordenada de latitude de onde o passeio de táxi terminou.
- passenger_count - número inteiro indicando o número de passageiros no trajeto de táxi.

3.Variável Alvo

- fare_amount - valor em dólar flutuante do custo do táxi. Este valor é apenas no conjunto de treinamento; isso é o que você está prevendo no conjunto de testes e é necessário em seu envio CSV.

Para esta DataSet, faremos algumas operações que tem por objetivo explicar o preço da corrida de taxi, separado em seções:

- Claning Data
- Análise Exploratória de Dados
- Modelagem Estatística
- Regressão Lienar
- Avançado

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from wordcloud import WordCloud
from collections import Counter
from sklearn import preprocessing
from datetime import datetime
import ast
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'wordcloud'

# Lendo e verificando os dados

In [ ]:
df = pd.read_csv("./data/train.csv", nrows = 1000000)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

# Entendendo a explicabilidade do DataSet (benchmark)

Rodar uma regressão linear no início da análise exploratória de dados nos permite entender o quão explicativas são as variáveis independentes iniciais, que servirá como referência ao final da nossa análise

In [ ]:
df.columns

In [ ]:
# rodando uma regressão inicial para servir de benchmark para a análise

formula = 'fare_amount ~ pickup_longitude + pickup_latitude + dropoff_longitude + dropoff_latitude + passenger_count'

result = sm.ols(formula, data=df).fit()
print(result.summary())

Podemos verificar que não existe relação nenhuma contida nas variáveis independentes que possam explicar as taxas de corridas de taxi cobradas.

# CLEANING DATA

Para melhor entendimento no processamento das colunas do DataSet, precisamos entender os valores que estão contidos em cada coluna e como faremos para confiar nos dados, ou seja, para estarmos seguros de que os números nas colunas são coerentes e estão dentro de um intervalo esperado.

In [ ]:
# Contagem de valores nulos
df.isnull().sum().sort_values(ascending=False)

## Eliminando os valores nulos

In [ ]:
# Dropando 10 observações com dados nulos
df = df.drop(df[df.isnull().any(1)].index, axis = 0)

In [ ]:
df.shape

## Verificando a variável alvo fare_amount

In [ ]:
df['fare_amount'].describe()

In [ ]:
from collections import Counter
Counter(df['fare_amount']<0)

Descobrimos que existem 38 valores negativos de _fare amount_ - como não podemos ter taxas de cobrança de corrida de taxi negativa, vamos eliminar estas linhas

In [ ]:
# Dropando 38 colunas com taxa de corrida negativa
df = df.drop(df[df['fare_amount']<0].index, axis=0)
df.shape

In [ ]:
df['fare_amount'].describe()

In [ ]:
df['fare_amount'].sort_values(ascending=False).nlargest(10)

## Verificando a variável passenger_count

In [ ]:
df['passenger_count'].describe()


In [ ]:
df['passenger_count'].sort_values(ascending=False).nlargest(5)

In [ ]:
df[df['passenger_count']>6]

Descobrimos que temos uma observação com 208 passageiros - vamos eliminar esta linha e ficar com um máximo de 6 passageiros por corrida.

In [ ]:
# Dropando uma linha com 608 passageiros
df = df.drop(df[df['passenger_count']==208].index, axis = 0)

In [ ]:
df['passenger_count'].describe()

## Verificando as variáveis de latitude

A latitude é um ângulo (definido abaixo) que varia de 0 ° no equador a 90 ° (norte ou sul) nos pólos. Linhas de latitude constante, ou paralelos, correm de leste a oeste como círculos paralelos ao equador. A latitude é usada junto com a longitude para especificar a localização precisa dos recursos na superfície da Terra.

Isso significa de não podemos ter valores de latitude menores que -90 e maiores que 90.

### pickup_latitude

In [ ]:
df['pickup_latitude'].describe()

In [ ]:
df[df['pickup_latitude']<-90]

In [ ]:
df[df['pickup_latitude']>90]

Percebemos que temos 3 observações com valores menores que -90 e 9 observações com valores maiores que 90 - vamos eliminar estas 12 linhas.

In [ ]:
# Dropando 12 linhas com valores fora do experado para pickup_latitude
df = df.drop(((df[df['pickup_latitude']<-90])|(df[df['pickup_latitude']>90])).index, axis=0)

In [ ]:
df[(df['pickup_latitude']<-90) & (df['pickup_latitude']>90)].count()

In [ ]:
df.shape

### dropoff_latitude

In [ ]:
df[df['dropoff_latitude']<-90]

In [ ]:
df[df['dropoff_latitude']>90]

Percebemos que temos uma observação com valor menos que -90 e 7 observações com valores maiores que 90 - vamos eliminar estas 8 linhas.

In [ ]:
# dropando 8 linhas com valores fora do experado para pickup_latitude
df = df.drop(((df[df['dropoff_latitude']<-90])|(df[df['dropoff_latitude']>90])).index, axis=0)

In [ ]:
df[(df['dropoff_latitude']<-90) & (df['dropoff_latitude']>90)].count()

In [ ]:
df.shape

## Verificando as variáveis de longitude

Longitude, algumas vezes representada pela letra grega λ (lambda), descreve a localização de um lugar na Terra medido em graus, de zero a 180 para leste ou para oeste, a partir do Meridiano de Greenwich.

Isso significa de não podemos ter valores de longitude menores que -180 e maiores que 180. 

### pickup_longitude

In [ ]:
df['pickup_longitude'].describe()

In [ ]:
df[df['pickup_longitude']<-180]

In [ ]:
df[df['pickup_longitude']>180]

Percebemos que temos 11 valores menores que -180 e nenhum valor maiores que 180 - vamos eliminar estas 11 linhas

In [ ]:
# Dropando 11 linhas com valores fora do experado para pickup_latitude
df = df.drop(((df[df['pickup_longitude']<-180])|(df[df['pickup_longitude']>180])).index, axis=0)

In [ ]:
df[(df['pickup_longitude']<-180) & (df['pickup_longitude']>180)].count()

### dropoff_longitude

In [ ]:
df[df['dropoff_longitude']<-180]

In [ ]:
df[df['dropoff_longitude']>180]

Percebemos que temos 9 observações com valor menores que -180 e nenhuma observação com valores maiores que 180 - vamos eliminar estas 9 linhas.

In [ ]:
# Dropando 11 linhas com valores fora do experado para pickup_latitude
df = df.drop(((df[df['dropoff_longitude']<-180])|(df[df['dropoff_longitude']>180])).index, axis=0)

In [ ]:
df[(df['dropoff_longitude']<-180) & (df['dropoff_longitude']>180)].count()

# ANÁLISE EXPLORATÓRIA DE DADOS - EDA (Exploratory Data Analysis)

Para a Análise exploratória de dados, devemos ter algumas considerações para orientar o trabalho de modelagem dos dados:

- Como o número de passageiros afeta o preço da corrida de taxi? 
- Como a data de pickup afeta o preço da corrida de taxi? 
- Como o horário de pickup afeta o preço da corrida de taxi? 
- Como o dia do mês e da semana de pickup afeta o preço da corrida de taxi?  
- Como a distância viajada afeta o preço da corrida de taxi?

Agora está claro que temos que trabalhar com as variáveis de pickup_datetime e distância para explicar o preço da corridda. A partir daí, entendemos que teremos que fazer transformações matemáticas 

### 1. Distância

Inicialmente, teremos que explorar as variáveis de latitude e longitude com o objetivo de obter algum valor para distância percorrida durante a corrida.

### 2. Dados da data e horário do pickup

Em seguida vamos fazer o split da variável pickup_datetime para obtenção dos valores de início da corrida de taxi.

# 1. DISTÂNCIA

<img src='img/haversine.PNG' width=200>

Utilizando so dados de latitude e longitude para o pickup e dropoff, vamos calcular a distância e chegar a nossas conclusões sobre como pickup_location afeta a tarifa, através da criação de uma nova coluna que armazene a distância entre o pickup e o drop.

Podemos calcular a distância em uma esfera quando as latitudes e longitudes são dadas pela fórmula de Haversine

haversine (θ) = sin² (θ / 2)

Esta fórmula está baseada nos valores conhecidos de latitude representados por φ e de longitude representados por λ, e do raio da Terra representado por R (raio médio = 6,371km) considerando as coordenadas inicial e final representadas por 1 e 2 neste caso.

a = sin² ((φ2 - φ1) / 2) + cos φ1. cos φ2. sin² ((λ2 - λ1) / 2)

c = 2 * atan² (√a, √ (1 − a))

d = R ⋅ c

d = distância Haversine

Consulte esta página para mais informações e exemplos sobre a fórmula de Haversine


## [Fórmula de Haversine](https://en.wikipedia.org/wiki/Haversine_formula)

_Da Wikipédia, a enciclopédia livre_

A fórmula de Haversine determina a distância do grande círculo entre dois pontos em uma esfera, dadas as suas longitudes e latitudes. Importante na navegação, é um caso especial de uma fórmula mais geral em trigonometria esférica, a lei de haversines, que relaciona os lados e ângulos dos triângulos esféricos.

A primeira tabela de haversines em inglês foi publicada por James Andrew em 1805, mas Florian Cajori credita um uso anterior por José de Mendoza y Ríos em 1801. O termo haversine foi cunhado em 1835 por James Inman.

Esses nomes decorrem do fato de que eles são costumeiramente escritos em termos da função haversina, dada por haversin (θ) = sin² (θ / 2). As fórmulas poderiam igualmente ser escritas em termos de qualquer múltiplo do haversine, como a antiga função versine (duas vezes o haversine). Antes do advento dos computadores, a eliminação da divisão e multiplicação por fatores de dois provou ser conveniente o bastante para incluir tabelas de valores e logaritmos haversinos nos textos de navegação e trigonométricos do século XIX e início do século XX. Atualmente, a forma haversina também é conveniente, pois não tem coeficiente na frente da função sin².

In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [df]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φ2 - φ1)/2) + cos φ1 . cos φ2 . sin²((λ2 - λ1)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude').head(10)

In [ ]:
df['H_Distance'].head(10)

In [ ]:
df.head()

# 2. DADOS DA DATA E HORÁRIO DO PICKUP 

Vamos fazer o split da variável pickup_datetime para a criação de novas colunas dentro do nosso DataSet que incluem:

- ano
- mês
- dia do mês
- dia da semana
- hora

In [ ]:
df.dtypes

In [ ]:
# Esta célula é lenta pra rodar
df['key'] = pd.to_datetime(df['key'])
df['pickup_datetime']  = pd.to_datetime(df['pickup_datetime'])

In [ ]:
df.dtypes

Now that we have calculated the distance, we shall create columns for the following -

- year
- month
- date
- hour
- day of week

In [ ]:
data = [df]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day_of_Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [ ]:
df.head()

## Explorando a variável passenger_count

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['passenger_count'], bins=15)
plt.xlabel('No. of Passengers')
plt.ylabel('Frequency')


In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df['passenger_count'], y=df['fare_amount'], s=1.5)
plt.xlabel('No. of Passengers')
plt.ylabel('Fare')

Dos 2 gráficos acima, podemos ver que os passageiros solteiros são os viajantes mais frequentes, e a tarifa mais alta também parece vir de táxis que transportam apenas 1 passageiro.

## Explorando a variável Date (dia do mês do pickup)


In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['Date'], bins=100)
plt.xlabel('Date')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df['Date'], y=df['fare_amount'], s=1.5)
plt.xlabel('Date')
plt.ylabel('Fare')

## Explorando a variável Hour (hora do pickup)

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['Hour'], bins=100)
plt.xlabel('Hour')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df['Hour'], y=df['fare_amount'], s=1.5)
plt.xlabel('Hour')
plt.ylabel('Fare')

Podemos entender a influência da frequência das corridas de taxi, sendo que os valores de mínimo estão às 5AM e os valores de máximo estão às 7PM.

## Explorando a variável Day of Week


In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['Day_of_Week'], bins=100)
plt.xlabel('Day_of_Week')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df['Day_of_Week'], y=df['fare_amount'], s=1.5)
plt.xlabel('Day_of_Week')
plt.ylabel('Fare')

A variável retorna o dia da semana de tal forma que assume-se que a semana começa na segunda-feira, que é denotada por 0 e termina no domingo, que é denotada por 6. Esse método está disponível em ambos os valores Series com data e hora (usando o acessador dt) ou DatetimeIndex.

As tarifas mais altas parecem estar no domingo e na segunda-feira, e as mais baixas na quarta e na sexta-feira. Talvez as pessoas viajem longas distâncias no domingo e segunda-feira (visitando a família e voltando para casa) e, portanto, as altas tarifas. Isto pode indicar que as pessoas acabam por ficar em casa em uma sexta-feira depois de uma semana de trabalho.

## Explorando a variável H_Distance 

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['H_Distance'], bins=100)
plt.xlabel('H_Distance')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(df['H_Distance'], y=df['fare_amount'], s=1.5)
plt.xlabel('H_Distance')
plt.ylabel('Frequency')

Em primeiro lugar, vamos verificar a frequência das distâncias que calculamos usando a fórmula de Haversine com a criação de intervalos (0-10 kms, 10-20 kms e assim por diante) para a verificação da frequência dos dados e eventuais outlires.

In [ ]:
bins_0 = df.loc[(df['H_Distance'] == 0), ['H_Distance']]
bins_1 = df.loc[(df['H_Distance'] > 0) & (df['H_Distance'] <= 10),['H_Distance']]
bins_2 = df.loc[(df['H_Distance'] > 10) & (df['H_Distance'] <= 50),['H_Distance']]
bins_3 = df.loc[(df['H_Distance'] > 50) & (df['H_Distance'] <= 100),['H_Distance']]
bins_4 = df.loc[(df['H_Distance'] > 100) & (df['H_Distance'] <= 200),['H_Distance']]
bins_5 = df.loc[(df['H_Distance'] > 200) & (df['H_Distance'] <= 300),['H_Distance']]
bins_6 = df.loc[(df['H_Distance'] > 300),['H_Distance']]

bins_0['bins'] = '0'
bins_1['bins'] = '0-10'
bins_2['bins'] = '11-50'
bins_3['bins'] = '51-100'
bins_4['bins'] = '100-200'
bins_5['bins'] = '201-300'
bins_6['bins'] = '>300'

dist_bins = pd.concat([bins_0,bins_1,bins_2,bins_3,bins_4,bins_5,bins_6])
#len(dist_bins)
dist_bins.columns

In [ ]:
dist_bins.head()

In [ ]:
dist_bins['bins'].value_counts()

Existem valores maiores que 100 kms,porém em Nova York precisamos explorar por que as pessoas pegam táxis para viajar mais de 100 quilômetros. Como a frequência do número de bins de 100 a 200 kms é bastante alto, a ideia será mantê-los. Esses outliers podem ser causados por erros de digitação ou valores ausentes na latitude ou longitude. Seguindo a análise exploratória e modelagem estatística, devemos pensar em remover campos dos seguintes:

- A latitude de pickup e a longitude de pickup são 0, mas a latitude e a longitude de dropoff não são 0, mas a tarifa é 0
- Vice-versa do ponto 1.
- A latitude de pickup e a longitude de pickup são 0, mas a latitude e longitude de dropoff não são 0, mas a tarifa é NÃO 0. Aqui teremos que imputar os valores de distância a serem calculados de acordo com fórmula de cobrança dos taxímetros.

## A latitude de pickup e a longitude de pickup são 0, mas a latitude e a longitude de dropoff não são 0, mas a tarifa é 0

In [ ]:
#pickup latitude and longitude = 0
df.loc[((df['pickup_latitude']==0) & (df['pickup_longitude']==0))&((df['dropoff_latitude']!=0) & (df['dropoff_longitude']!=0)) & (df['fare_amount']==0)]

In [ ]:
df = df.drop(df.loc[((df['pickup_latitude']==0) & (df['pickup_longitude']==0))&\
                    ((df['dropoff_latitude']!=0) & (df['dropoff_longitude']!=0)) & (df['fare_amount']==0)].index, axis=0)

# 1 row dropped
df.shape

In [ ]:
#dropoff latitude and longitude = 0
df.loc[((df['pickup_latitude']!=0) & (df['pickup_longitude']!=0))&((df['dropoff_latitude']==0) & (df['dropoff_longitude']==0)) & (df['fare_amount']==0)]

In [ ]:
df = df.drop(df.loc[((df['pickup_latitude']!=0) & (df['pickup_longitude']!=0))&\
                    ((df['dropoff_latitude']==0) & (df['dropoff_longitude']==0)) & (df['fare_amount']==0)].index, axis=0)

# 3 rows dropped
df.shape

## Valores de distância maiores do que 200Km

Verificando os campos H_Distance que são maiores que 200 kms, pois dificilmente as pessoas podem viajar mais de 200 kms no máximo em NYC em um taxi.

In [ ]:
high_distance = df.loc[(df['H_Distance']>200)&(df['fare_amount']!=0)]

In [ ]:
high_distance.head()

In [ ]:
high_distance.fare_amount.hist(bins=200)

In [ ]:
high_distance.shape

Como podemos ver no DF acima, as distâncias anormalmente altas se devem ao fato de as coordenadas de pickup ou dropoff estarem incorretas ou 0. No entanto, como todos esses valores têm tarifas, descartá-las não é a melhor estratégia, pois elas contêm dados cruciais. Em vez disso, vamos substituir os valores iniciais de distância por valores de distância calculados usando a tarifa usando a fórmula de cobrança de carridas de taxi abaixo.

#### distance = (fare_amount - 2.5)/1.56

In [ ]:
high_distance['H_Distance'] = high_distance.apply(lambda row: (row['fare_amount'] - 2.50)/1.56, axis=1)


In [ ]:
# Verificando a substituição dos dadso de distância calculados de acordo com a fórmula 
high_distance.head()

In [ ]:
# Sincronizando os dados com os valores calculados no DataFrame original 
df.update(high_distance)

In [ ]:
df.shape

## Verificando os valores com H_Distance iguais a zero

Agora vamos verificar as linhas onde os valores de distância são 0

In [ ]:
df[df['H_Distance']==0].head()

In [ ]:
df[df['H_Distance']==0].shape

Podemos ver algumas linhas com distância = 0. Isso pode ser devido a 2 motivos

- O táxi esperou o tempo todo e o passageiro acabou cancelado. É por isso que as coordenadas de coleta e queda são as mesmas e talvez o passageiro tenha sido cobrado pelo tempo de espera.

- As coordenadas de coleta de dropoff não foram inseridas. Em outras palavras, esses valores estão faltando.

28667 linhas são muitas linhas para serem excluídas. Precisamos imputar esses valores ausentes. Podemos adotar a estratégia de imputar os valores de distância perdidos com a tarifa e o preço médio por quilômetro dos táxis de Nova York.

UVerificando as fórmulas de calculo de tarifa temos:

#### Preço base de USD 2,5 + USD 1,56 / km -> das 06:00 h às 20:00 h de segunda a sexta

#### Preço base de USD 3.0 + USD 1,56 / km -> das 20:00 h às 06:00 h de segunda a sexta e sáb e dom

No entanto, antes de prosseguirmos com as etapas acima, vamos verificar os cenários a seguir para imputar o valor da tarifa ausente e a H_Distance nos dados

1. Tarifa e Distância são ambas 0. 
2. A tarifa não é 0 e é menor que o valor base, mas a distância é 0.
3. A tarifa é 0, mas a distância não é 0.
4. A tarifa não é 0, mas a distância é 0.

## 1. Tarifa e Distância são ambas 0. 

De acordo com a tabela acima, nós as excluiremos, pois elas não nos fornecem nenhuma informação com relação aos dados.

In [ ]:
df[(df['H_Distance']==0)&(df['fare_amount']==0)]

In [ ]:
df = df.drop(df[(df['H_Distance']==0)&(df['fare_amount']==0)].index, axis = 0)

In [ ]:
# 4 linhas excluidas
df[(df['H_Distance']==0)].shape

## 2. A tarifa não é 0 e é menor que o valor base, mas a distância é 0.

A ideia inicial é excluir essas linhas, pois o mínimo é de US $ 2,50 e essas tarifas são valores incorretos.

#### Entre 6AM e 8PM em dias de semana Mon-Fri

In [ ]:
rush_hour = df.loc[(((df['Hour']>=6)&(df['Hour']<=20)) & ((df['Day_of_Week']>=1) & (df['Day_of_Week']<=5)) & (df['H_Distance']==0) & (df['fare_amount'] < 2.5))]
rush_hour

In [ ]:
# Vamos excluir estas duas linhas pois estão abaixo do limite inicial
df=df.drop(rush_hour.index, axis=0)

In [ ]:
df.shape

#### Entre 8PM e 6AM em dias de semana Mon-Fri

In [ ]:
non_rush_hour = df.loc[(((df['Hour']<6)|(df['Hour']>20)) & ((df['Day_of_Week']>=1)&(df['Day_of_Week']<=5)) & (df['H_Distance']==0) & (df['fare_amount'] < 3.0))]
#print(Counter(non_work_hours['Hour']))
#print(Counter(non_work_hours['Day of Week']))
non_rush_hour.head()

In [ ]:
non_rush_hour.fare_amount.value_counts()


Vamos manter esses valores uma vez que, como o fare_amount não é <2,5 (que é a tarifa básica), esses valores parecem legítimos.

#### Saturday and Sunday all hours

In [ ]:
weekends = df.loc[((df['Day_of_Week']==0) | (df['Day_of_Week']==6)) & (df['H_Distance']==0) & (df['fare_amount'] < 3.0)]
weekends.head()
#Counter(weekends['Day of Week'])

In [ ]:
weekends.fare_amount.value_counts()

Vamos manter esses valores também pois novamente o fare_amount não é <2,5 (que é a tarifa básica), de novo esses valores parecem legítimos.

## 3. A tarifa é 0, mas a distância não é 0.

Esses valores precisam ser imputados.

devemos calcular a tarifa correspondente baseado da distância. Devemos utilizar a seguinte fórmula (para simplificar em relação a horas de não pressa e fins de semana)

#### tarifa = 2.5 + 1.56 (H_Distance)

In [ ]:
scenario_3 = df.loc[(df['H_Distance']!=0) & (df['fare_amount']==0)]
scenario_3.shape

In [ ]:
scenario_3.fare_amount.value_counts()

In [ ]:
scenario_3['fare_amount'] = scenario_3.apply(lambda row: ((row['H_Distance'] * 1.56) + 2.50), axis=1)

In [ ]:
scenario_3['fare_amount']

In [ ]:
df.update(scenario_3)

In [ ]:
df.shape

## 4. A tarifa não é 0, mas a distância é 0.

Esses valores precisam ser imputados.

In [ ]:
scenario_4 = df.loc[(df['H_Distance']==0) & (df['fare_amount']!=0)]
scenario_4.head()

In [ ]:
scenario_4.shape

Levando em consideração a fórmula básica:

- Preço base de USD 2,5 + USD 1,56 / km -> das 06: 00h às 20: 00h, de segunda a sexta-feira;

- Preço base de USD 3,0 + USD 1,56 / km -> 20:00 h às 06:00 h de segunda a sexta e sáb e dom

Devemos adotar cálculos de distância como:

- distância = (tarifa - 2,5) / 1,56 -> 6:00 às 20:00 seg - sex (hora do rush)

- distância = (tarifa - 3.0) / 1.56 -> das 8h às 6h seg-sex e sáb e dom (hora do rush)

### Rush Hour

In [ ]:
rush_hour_4 = scenario_4.loc[(((scenario_4['Hour']>=6)&(scenario_4['Hour']<=20)) & ((scenario_4['Day_of_Week']>=1) & (scenario_4['Day_of_Week']<=5)))]
rush_hour_4.shape

In [ ]:
rush_hour_4.head()

In [ ]:
rush_hour_4['H_Distance'] = rush_hour_4.apply(lambda row: (row['fare_amount'] - 2.5) / 1.56, axis=1)
rush_hour_4.head()

In [ ]:
df.update(rush_hour_4)

### Non Rush Hour

In [ ]:
scenario_4.shape[0] - rush_hour_4.shape[0]

In [ ]:
non_rush_hour_4 = scenario_4.drop(rush_hour_4.index, axis=0)
non_rush_hour_4.shape

In [ ]:
non_rush_hour_4['H_Distance'] = non_rush_hour_4.apply(lambda row: (row['fare_amount'] - 3.0) / 1.56, axis=1)
non_rush_hour_4.head()

In [ ]:
df.update(non_rush_hour_4)

# MODELAGEM ESTATÍSTICA

Agora vamos criar uma cópia do DataFrame original e entender como podemos explicar os valores de taxa de corrida de taxi com os dados criados.

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.shape

In [ ]:
df_copy.columns

In [ ]:
df_copy.info()

In [ ]:
df_copy.head()

## Normalizando os dados

Vamos aplicar o StandardScaler() para termos todos os dados com a mesma ordem de grandeza

In [ ]:
df_copy.drop(columns=['key','pickup_datetime', 'pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'],axis=1,inplace=True)

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_copy)

In [ ]:
df_scaled = pd.DataFrame(scaled_data,columns=['fare_amount', 'passenger_count', 'H_Distance', 'Year', 'Month', 'Date','Day_of_Week', 'Hour'])
df_scaled.head()

In [ ]:
formula = 'fare_amount ~ passenger_count + H_Distance + Year + Month + Date + Day_of_Week + Hour'

result = sm.ols(formula, data=df_scaled).fit()
print(result.summary())

## Visualização dos dados e estratégias de transformação 

In [ ]:
df_copy.columns

In [ ]:
corr = df_copy.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
sns.pairplot(df_copy.sample(n=10000))

Podemos visualizar que existe uma relação linear significativa entre a taxa da corrida e a distância percorrida (talvez impulsionada pela inserção de valores de acrodo com a fóormula de cálculo do preço). Além disso, também podemos observar que ambas as variávais tem assimeria positiva presente nos gráficos de distribuição. Portanto a melhor estratégia é transformar ambas as variáveis com o LOG.

In [ ]:
formula = 'np.log1p(fare_amount) ~ passenger_count + np.log1p(H_Distance) + Year + Month + Date + Day_of_Week + Hour'

result = sm.ols(formula, data=df_copy).fit()
print(result.summary())

# AVANÇADO

Vamos aplicar as análises e modelagens para os dados de teste e verificar a qualidade de predição do modelo

In [ ]:
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split
from keras import layers
from keras import models

from keras import regularizers
from keras.metrics import mean_squared_logarithmic_error


In [ ]:
test = pd.read_csv("./data/test.csv", nrows = 1000000)
test.head()

## Verificação dos dados

In [ ]:
test.isnull().sum().sort_values(ascending=False)

In [ ]:
test.passenger_count.value_counts()

In [ ]:
test.pickup_latitude.describe()

In [ ]:
test.dropoff_latitude.describe()

In [ ]:
test.pickup_longitude.describe()

In [ ]:
test.dropoff_longitude.describe()

## Cálculo das distâncias

In [ ]:
def haversine_test(lat1, long1, lat2, long2):
    data = [test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φ2 - φ1)/2) + cos φ1 . cos φ2 . sin²((λ2 - λ1)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

haversine_test('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude').head(10)
test.head()

In [ ]:
test.H_Distance.describe()

In [ ]:
test.H_Distance.value_counts().nlargest(4)

## Transformação das variáveis Date and Time

In [ ]:
test['key'] = pd.to_datetime(test['key'])
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])

In [ ]:
data = [test]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day_of_Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [ ]:
test.head()

## Excluíndo as colunas que não interessam

In [ ]:
test.drop(columns=['key','pickup_datetime', 'pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'],axis=1,inplace=True)

## Fazendo as predições

In [ ]:


df_copy['fare_amount'] = np.log1p(df_copy['fare_amount'])
df_copy['H_Distance'] = np.log1p(df_copy['H_Distance'])

test['H_Distance'] = np.log1p(test['H_Distance'])



In [ ]:
x_train = df_copy.drop(['fare_amount'],axis=1)
y_train = df_copy.fare_amount

x_test = test

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(x_train,y_train)
y_pred_lr = model_lr.predict(x_test)
y_pred_lr = np.exp(y_pred_lr)

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['fare_amount'] = y_pred_lr
submission.to_csv('submission_lr.csv', index=False)
submission.head(20)

In [ ]:
#!pip install ml_metrics

In [ ]:
from ml_metrics import rmse

y_pred_lr_train = model_lr.predict(x_train)

rmse(y_train, y_pred_lr_train)

# LGBM

In [ ]:
#!pip install lightgbm

In [ ]:
import lightgbm as lgbm

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1     
    }


In [ ]:
pred_test_y = np.zeros(x_test.shape[0])
pred_test_y.shape

In [ ]:
train_set = lgbm.Dataset(x_train, y_train, silent=True)
train_set

In [ ]:
model = lgbm.train(params, train_set = train_set, num_boost_round=300)

In [ ]:
print(model)

In [ ]:
pred_test_y = model.predict(x_test, num_iteration = model.best_iteration)
pred_test_y = np.exp(pred_test_y)

In [ ]:
submission['fare_amount'] = pred_test_y
submission.to_csv('submission_LGB.csv', index=False)
submission.head(20)

In [ ]:
y_pred_lgbm_train = model.predict(x_train)

rmse(y_train, y_pred_lgbm_train)

# XGBOOST

In [ ]:
#!pip install xgboost

In [ ]:
import xgboost as xgb 

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test)

In [ ]:
#set parameters for xgboost
params = {'max_depth':7,
          'eta':1,
          'silent':1,
          'objective':'reg:linear',
          'eval_metric':'rmse',
          'learning_rate':0.05
         }
num_rounds = 50

In [ ]:
xb = xgb.train(params, dtrain, num_rounds)

In [ ]:
y_pred_xgb = xb.predict(dtest)
y_pred_xgb = np.exp(y_pred_xgb)
print(y_pred_xgb)

In [ ]:
submission['fare_amount'] = y_pred_xgb
submission.to_csv('submission_XGB.csv', index=False)
submission.head(20)

In [ ]:
y_pred_xb_train = xb.predict(dtrain)
rmse(y_train, y_pred_xb_train)

O melhor _Score_ no kaggle foi de 3.98717 com LGBM, porém ficou muito próximo do XGBOOST (4.07935), porém sem a aplicação da função LOG.
